# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts you have learned so far, from adding interactions and polynomials to your model to regularization!

## Summary

You will be able to:

- Build a linear regression model with interactions and polynomial features 
- Use feature selection to obtain the optimal subset of features in a dataset

## Let's Get Started!

Below we import all the necessary packages for this lab.

In [1]:
# Run this cell without changes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

Load the data.

In [2]:
# Run this cell without changes

# Load data from CSV
df = pd.read_csv("ames.csv")
# Subset columns
df = df[['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd',
         'GarageArea', 'Fireplaces', 'SalePrice']]

# Split the data into X and y
y = df['SalePrice']
X = df.drop(columns='SalePrice')

# Split into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=0)

## Build a Baseline Housing Data Model

Above, we imported the Ames housing data and grabbed a subset of the data to use in this analysis.

Next steps:

- Scale all the predictors using `StandardScaler`, then convert these scaled features back into DataFrame objects
- Build a baseline `LinearRegression` model using *scaled variables* as predictors and use the $R^2$ score to evaluate the model 

In [3]:
# Your code here

# Scale X_train and X_val using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
# Ensure X_train and X_val are scaled DataFrames
# (hint: you can set the columns using X.columns)
X_train = pd.DataFrame(X_train_scaled, columns=X.columns)
x_val = pd.DataFrame(X_val_scaled, columns=X.columns)

X_train


,LotArea,OverallQual,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,Fireplaces
0,-0.114710,-0.099842,-0.509252,-0.639316,-0.804789,1.261552,0.499114,0.250689,0.327629,-0.994820
1,-0.176719,0.632038,-0.509252,0.838208,0.641608,-0.808132,-0.247249,-0.365525,0.079146,-0.994820
2,-0.246336,-0.831723,1.304613,-0.012560,-0.329000,-0.808132,-0.944766,-0.981739,-1.105931,-0.994820
3,-0.378617,-0.831723,1.304613,-0.339045,-0.609036,-0.808132,-1.146010,-0.981739,-1.134602,0.588023
4,-0.010898,-1.563603,1.304613,-2.531499,-1.315922,0.550523,-0.481708,0.250689,-2.281450,-0.994820
...,...,...,...,...,...,...,...,...,...,...
816,-0.532331,-0.099842,-0.509252,-0.510628,-0.897228,-0.808132,-1.353116,-2.214167,-0.274466,0.588023
817,-0.309245,-0.099842,-0.509252,0.514106,0.315353,-0.808132,-0.481708,-0.365525,0.088703,-0.994820
818,0.119419,0.632038,-0.509252,-0.513011,-0.899947,1.684999,0.796096,0.866903,-0.207566,0.588023
819,-0.002718,-0.099842,1.304613,-0.889542,-1.329516,0.783758,-0.290233,-0.365525,-0.852668,-0.994820


In [4]:
# Your code here

# Create a LinearRegression model and fit it on scaled training data
linreg = LinearRegression()
linreg.fit(X_train, y_train)
# Calculate a baseline r-squared score on training data
baseline_train = linreg.score(X_train, y_train)
baseline_val = linreg.score(X_val, y_val)

baseline_train, baseline_val

(0.7868344817421309, -2301787.8249637415)

## Add Interactions

Instead of adding all possible interaction terms, let's try a custom technique. We are only going to add the interaction terms that increase the $R^2$ score as much as possible. Specifically we are going to look for the 7 interaction terms that each cause the most increase in the coefficient of determination.

### Find the Best Interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Create a data structure that stores the pair of columns used as well as the $R^2$ score for each combination.

***Hint:*** We have imported the `combinations` function from `itertools` for you ([documentation here](https://docs.python.org/3/library/itertools.html#itertools.combinations)). Try applying this to the columns of `X_train` to find all of the possible pairs.

Print the 7 interactions that result in the highest $R^2$ scores.

In [5]:
# Your code here

# Set up data structure
interactions = []

# Find combinations of columns and loop over them
column_pairs = list(combinations(X_train.columns, 2))
for (col1, col2) in column_pairs:
    # Make copies of X_train and X_val
    features_train = X_train.copy()
    features_val = X_val.copy()
    
    # Add interaction term to data
    features_train['interaction'] = features_train[col1] * features_train[col2]
    features_val['interaction'] = features_val[col1] * features_val[col2]
    
    # Find r-squared score (fit on training data, evaluate on validation data)
    score = LinearRegression().fit(features_train, y_train).score(features_val, y_val)

    
    # Append to data structure
    interactions.append(((col1, col2), score))
    
# Sort and subset the data structure to find the top 7
top_7 = sorted(interactions, key=lambda record: record[1], reverse=True)[:7]
print("Top 7 interactions:", top_7)


Top 7 interactions: [(('1stFlrSF', 'Fireplaces'), -1729473.9372425335), (('TotalBsmtSF', 'Fireplaces'), -1882649.7448176094), (('GarageArea', 'Fireplaces'), -2002405.8581155958), (('OverallQual', 'TotRmsAbvGrd'), -2060919.6666263596), (('TotRmsAbvGrd', 'Fireplaces'), -2084329.697906163), (('OverallQual', 'Fireplaces'), -2245515.610545067), (('GrLivArea', 'Fireplaces'), -2250938.4385458687)]


### Add the Best Interactions

Write code to include the 7 most important interactions in `X_train` and `X_val` by adding 7 columns. Use the naming convention `"col1_col2"`, where `col1` and `col2` are the two columns in the interaction.

In [6]:
# Your code here

# Loop over top 7 interactions
for record in top_7:
    # Extract column names from data structure
    col1, col2 = record[0]
    # Construct new column name
    new_col_name = col1 + "_" + col2
    # Add new column to X_train and X_val
    X_train[new_col_name] = X_train[col1] * X_train[col2]
    X_val[new_col_name] = X_val[col1] * X_val[col2]

X_train

,LotArea,OverallQual,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,Fireplaces,1stFlrSF_Fireplaces,TotalBsmtSF_Fireplaces,GarageArea_Fireplaces,OverallQual_TotRmsAbvGrd,TotRmsAbvGrd_Fireplaces,OverallQual_Fireplaces,GrLivArea_Fireplaces
0,-0.114710,-0.099842,-0.509252,-0.639316,-0.804789,1.261552,0.499114,0.250689,0.327629,-0.994820,0.800620,0.636004,-0.325932,-0.025029,-0.249390,0.099325,-0.496529
1,-0.176719,0.632038,-0.509252,0.838208,0.641608,-0.808132,-0.247249,-0.365525,0.079146,-0.994820,-0.638285,-0.833866,-0.078736,-0.231026,0.363632,-0.628764,0.245968
2,-0.246336,-0.831723,1.304613,-0.012560,-0.329000,-0.808132,-0.944766,-0.981739,-1.105931,-0.994820,0.327296,0.012495,1.100202,0.816535,0.976654,0.827414,0.939872
3,-0.378617,-0.831723,1.304613,-0.339045,-0.609036,-0.808132,-1.146010,-0.981739,-1.134602,0.588023,-0.358127,-0.199366,-0.667173,0.816535,-0.577286,-0.489072,-0.673881
4,-0.010898,-1.563603,1.304613,-2.531499,-1.315922,0.550523,-0.481708,0.250689,-2.281450,-0.994820,1.309105,2.518386,2.269632,-0.391978,-0.249390,1.555503,0.479213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816,-0.532331,-0.099842,-0.509252,-0.510628,-0.897228,-0.808132,-1.353116,-2.214167,-0.274466,0.588023,-0.527591,-0.300261,-0.161392,0.221068,-1.301982,-0.058710,-0.795664
817,-0.309245,-0.099842,-0.509252,0.514106,0.315353,-0.808132,-0.481708,-0.365525,0.088703,-0.994820,-0.313720,-0.511443,-0.088243,0.036495,0.363632,0.099325,0.479213
818,0.119419,0.632038,-0.509252,-0.513011,-0.899947,1.684999,0.796096,0.866903,-0.207566,0.588023,-0.529190,-0.301663,-0.122054,0.547915,0.509759,0.371653,0.468123
819,-0.002718,-0.099842,1.304613,-0.889542,-1.329516,0.783758,-0.290233,-0.365525,-0.852668,-0.994820,1.322629,0.884934,0.848252,0.036495,0.363632,0.099325,0.288730


## Add Polynomials

Now let's repeat that process for adding polynomial terms.

### Find the Best Polynomials

Try polynomials of degrees 2, 3, and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of degree 4 with `PolynomialFeatures`, the particular column is raised to the power of 2 and 3 as well in other terms.

We only want to include "pure" polynomials, so make sure no interactions are included.

Once again you should make a data structure that contains the values you have tested. We recommend a list of tuples of the form:

`(col_name, degree, R2)`, so eg. `('OverallQual', 2, 0.781)` 

In [13]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np

# Set up data structure
polynomials = []

# Create an imputer
imputer = SimpleImputer(strategy='mean')

# Loop over all columns
for col in X_train.columns:
    # Loop over degrees 2, 3, 4
    for degree in (2, 3, 4):
        
        # Make a copy of X_train and X_val
        features_train = X_train.copy()
        features_val = X_val.copy()
    
        # Instantiate PolynomialFeatures with relevant degree
        poly = PolynomialFeatures(degree, include_bias=False)
        
        # Fit polynomial to column and transform column
        col_transformed_train = pd.DataFrame(poly.fit_transform(features_train[[col]]), index=features_train.index)
        col_transformed_val = pd.DataFrame(poly.transform(features_val[[col]]), index=features_val.index)
        
        # Rename columns to ensure they're strings
        col_transformed_train.columns = [f"{col}_poly_{i}" for i in range(degree)]
        col_transformed_val.columns = [f"{col}_poly_{i}" for i in range(degree)]
        
        # Convert all column names to strings
        features_train.columns = features_train.columns.astype(str)
        features_val.columns = features_val.columns.astype(str)
        
        # Add polynomial to data
        features_train = pd.concat([features_train.drop(col, axis=1), col_transformed_train], axis=1)
        features_val = pd.concat([features_val.drop(col, axis=1), col_transformed_val], axis=1)
    
        # Impute missing values
        features_train_imputed = pd.DataFrame(imputer.fit_transform(features_train), columns=features_train.columns, index=features_train.index)
        features_val_imputed = pd.DataFrame(imputer.transform(features_val), columns=features_val.columns, index=features_val.index)
    
        # Align indices
        common_train_idx = features_train_imputed.index.intersection(y_train.index)
        common_val_idx = features_val_imputed.index.intersection(y_val.index)
        
        features_train_clean = features_train_imputed.loc[common_train_idx]
        y_train_clean = y_train.loc[common_train_idx]
        features_val_clean = features_val_imputed.loc[common_val_idx]
        y_val_clean = y_val.loc[common_val_idx]
    
        # Find r-squared score
        score = LinearRegression().fit(features_train_clean, y_train_clean).score(features_val_clean, y_val_clean)
    
        # Append to data structure
        polynomials.append((col, degree, score))
    
# Sort and subset the data structure to find the top 7
top_7_polynomials = sorted(polynomials, key=lambda record: record[-1], reverse=True)[:7]
print("Top 7 polynomials:")
print(top_7_polynomials)

Top 7 polynomials:
[('OverallQual_Fireplaces', 3, -616451.4226592197), ('OverallQual', 4, -629630.1507210187), ('OverallQual', 3, -633606.9466754809), ('Fireplaces', 4, -634422.9699739632), ('Fireplaces', 3, -634426.0811938854), ('Fireplaces', 2, -659077.0827317967), ('OverallQual_Fireplaces', 2, -676662.2893447005)]


### Add the Best Polynomials

If there are duplicate column values in the results above, don't add multiple of them to the model, to avoid creating duplicate columns. (For example, if column `A` appeared in your list as both a 2nd and 3rd degree polynomial, adding both would result in `A` squared being added to the features twice.) Just add in the polynomial that results in the highest R-Squared.

In [19]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd

# Convert to DataFrame for easier manipulation
top_polynomials = pd.DataFrame(top_7_polynomials, columns=["Column", "Degree", "R^2"])

# Drop duplicate columns based on Column name
top_polynomials.drop_duplicates(subset="Column", inplace=True)

# Create an imputer
imputer = SimpleImputer(strategy='mean')

# Loop over remaining results
for col, degree, _ in top_polynomials.values:
    # Impute missing values
    X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train[[col]]), columns=[col], index=X_train.index)
    X_val_imputed = pd.DataFrame(imputer.transform(X_val[[col]]), columns=[col], index=X_val.index)
    
    # Create polynomial terms
    poly = PolynomialFeatures(degree, include_bias=False)
    train_poly = pd.DataFrame(
        poly.fit_transform(X_train_imputed),
        columns=poly.get_feature_names_out([col]),
        index=X_train.index
    )
    val_poly = pd.DataFrame(
        poly.transform(X_val_imputed),
        columns=poly.get_feature_names_out([col]),
        index=X_val.index
    )
    
    # Concat back to original
    X_train = pd.concat([X_train.drop(col, axis=1), train_poly], axis=1)
    X_val = pd.concat([X_val.drop(col, axis=1), val_poly], axis=1)

print("New shape of X_train:", X_train.shape)
print("New shape of X_val:", X_val.shape)

New shape of X_train: (821, 27)
New shape of X_val: (488, 27)


Check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [20]:
# Your code here
X_train.head()

,LotArea,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,1stFlrSF_Fireplaces,TotalBsmtSF_Fireplaces,...,OverallQual_Fireplaces^2,OverallQual_Fireplaces^3,OverallQual,OverallQual^2,OverallQual^3,OverallQual^4,Fireplaces,Fireplaces^2,Fireplaces^3,Fireplaces^4
0,-0.114710,-0.509252,-0.639316,-0.804789,1.261552,0.499114,0.250689,0.327629,0.800620,0.636004,...,0.009865,0.000980,-0.099842,0.009968,-0.000995,0.000099,-0.994820,0.989667,-0.984540,0.979440
1,-0.176719,-0.509252,0.838208,0.641608,-0.808132,-0.247249,-0.365525,0.079146,-0.638285,-0.833866,...,0.395344,-0.248578,0.632038,0.399472,0.252481,0.159578,-0.994820,0.989667,-0.984540,0.979440
2,-0.246336,1.304613,-0.012560,-0.329000,-0.808132,-0.944766,-0.981739,-1.105931,0.327296,0.012495,...,0.684614,0.566460,-0.831723,0.691762,-0.575354,0.478535,-0.994820,0.989667,-0.984540,0.979440
3,-0.378617,1.304613,-0.339045,-0.609036,-0.808132,-1.146010,-0.981739,-1.134602,-0.358127,-0.199366,...,0.239192,-0.116982,-0.831723,0.691762,-0.575354,0.478535,0.588023,0.345772,0.203322,0.119558
4,-0.010898,1.304613,-2.531499,-1.315922,0.550523,-0.481708,0.250689,-2.281450,1.309105,2.518386,...,2.419591,3.763681,-1.563603,2.444854,-3.822780,5.977310,-0.994820,0.989667,-0.984540,0.979440


## Full Model R-Squared

Check out the $R^2$ of the full model with your interaction and polynomial terms added. Print this value for both the train and validation data.

In [29]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2, so validation set is 20% of original data

In [31]:
# Your code here

lr = LinearRegression()
lr.fit(X_train, y_train)

print("Train R2:", lr.score(X_train, y_train))
print("Validation R2:", lr.score(X_val, y_val))
print("Test R2:", lr.score(X_test, y_test))


Train R2: 0.7436408980600209
Validation R2: 0.7868782847573534
Test R2: 0.7878986301832595


It looks like we may be overfitting some now. Let's try some feature selection techniques.

## Feature Selection

First, test out `RFE` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)) with several different `n_features_to_select` values. For each value, print out the train and validation $R^2$ score and how many features remain.

In [32]:
# Your code here
for n in [5, 10, 15, 20, 25]:
    rfe = RFE(LinearRegression(), n_features_to_select=n)
    X_rfe_train = rfe.fit_transform(X_train, y_train)
    X_rfe_val = rfe.transform(X_val)

    lr = LinearRegression()
    lr.fit(X_rfe_train, y_train)

    print("Train R^2:", lr.score(X_rfe_train, y_train))
    print("Test R^2: ", lr.score(X_rfe_val, y_val))
    print(f"Using {n} out of {X_train.shape[1]} features")
    print()

Train R^2: 0.7129424712747777
Test R^2:  0.7327039823201675
Using 5 out of 10 features

Train R^2: 0.7436408980600209
Test R^2:  0.7868782847573534
Using 10 out of 10 features

Train R^2: 0.7436408980600209
Test R^2:  0.7868782847573534
Using 15 out of 10 features

Train R^2: 0.7436408980600209
Test R^2:  0.7868782847573534
Using 20 out of 10 features

Train R^2: 0.7436408980600209
Test R^2:  0.7868782847573534
Using 25 out of 10 features



Now test out `Lasso` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)) with several different `alpha` values.

In [33]:
# Your code here
for alpha in [1, 10, 100, 1000, 10000]:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)

    print("Train R^2:", lasso.score(X_train, y_train))
    print("Validation R^2: ", lasso.score(X_val, y_val))
    print(f"Using { 26 - (sum(abs(lasso.coef_) < 10**(-10)))} out of {X_train.shape[1]} features")
    print("and an alpha of", alpha)
    print()

Train R^2: 0.7436408970459523
Validation R^2:  0.7868792609246269
Using 26 out of 10 features
and an alpha of 1

Train R^2: 0.7436407999750654
Validation R^2:  0.7868900582109771
Using 26 out of 10 features
and an alpha of 10

Train R^2: 0.7436310676380495
Validation R^2:  0.7869910954852047
Using 26 out of 10 features
and an alpha of 100

Train R^2: 0.7426575886646019
Validation R^2:  0.7873078645406397
Using 26 out of 10 features
and an alpha of 1000

Train R^2: 0.7225260733436096
Validation R^2:  0.7726470585701344
Using 22 out of 10 features
and an alpha of 10000



Compare the results. Which features would you choose to use?

In [ ]:
# Your written answer here
"""
The 17 features selected by Lasso with alpha 1000. 
This provides a good balance between model complexity (number of features) and performance (R^2 scores).

"""

## Evaluate Final Model on Test Data

### Data Preparation

At the start of this lab, we created `X_test` and `y_test`. Prepare `X_test` the same way that `X_train` and `X_val` have been prepared. This includes scaling, adding interactions, and adding polynomial terms.

In [40]:

# Scale X_test
X_test_scaled = scaler.transform(X_test)
X_test = pd.DataFrame(X_test_scaled, columns=X.columns)

# Add interactions to X_test
for record in top_7:
    col1, col2 = record[0]
    new_col_name = col1 + "_" + col2
    X_test[new_col_name] = X_test[col1] * X_test[col2]

# Add polynomials to X_val
for (col, degree, _) in top_polynomials.values:
    poly = PolynomialFeatures(degree, include_bias=False)
    poly_test = pd.DataFrame(poly.fit_transform(X_test[[col]]),
                                        columns=poly.get_feature_names([col]))
    X_test = pd.concat([X_test.drop(col, axis=1), poly_test], axis=1)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- 1stFlrSF_Fireplaces
- GarageArea_Fireplaces
- GrLivArea_Fireplaces
- OverallQual_Fireplaces
- OverallQual_TotRmsAbvGrd
- ...


### Evaluation

Using either `RFE` or `Lasso`, fit a model on the complete train + validation set, then find R-Squared and MSE values for the test set.

In [41]:
# Your code here
final_model = Lasso(alpha=10000)
final_model.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]))

print("R-Squared:", final_model.score(X_test, y_test))
print("MSE:", mean_squared_error(y_test, final_model.predict(X_test)))

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- 1stFlrSF_Fireplaces
- GarageArea_Fireplaces
- GrLivArea_Fireplaces
- OverallQual_Fireplaces
- OverallQual_TotRmsAbvGrd
- ...


## Level Up Ideas (Optional)

### Create a Lasso Path

From this section, you know that when using `Lasso`, more parameters shrink to zero as your regularization parameter goes up. In scikit-learn there is a function `lasso_path()` which visualizes the shrinkage of the coefficients while $alpha$ changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for Subset Selection

This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Ames housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to apply concepts of bias-variance tradeoff using extensions to linear models and feature selection.